# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 01.10.2023

Мягкий дедлайн: 23:59MSK 15.10.2023

Жесткий дедлайн: 23:59MSK 20.10.2023

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

Итогова оценка считается как
$$
min(task_1, task_2)*0.8 + max(task_1, task_2)*0.2
$$

где task_1 и task_2 - оценки за первое и второе заданиее соответсвенно.
Также, за домашнее задание выставляется 0, если не сделано нулевое или третье задание.
### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [1]:
import random

import torch
import numpy
import pandas
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2023-10-08 19:38:52--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data.txt.zip’

data.txt.zip            [               <=>  ] 201.24M  57.4MB/s    in 3.8s    

2023-10-08 19:38:56 (52.4 MB/s) - ‘data.txt.zip’ saved [211011981]



In [ ]:
df = pandas.read_csv('data.txt.zip', header=None)
display(df)

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515340,2006,51.28467,45.88068,22.19582,-5.53319,-3.61835,-16.36914,2.12652,5.18160,-8.66890,...,4.81440,-3.75991,-30.92584,26.33968,-5.03390,21.86037,-142.29410,3.42901,-41.14721,-15.46052
515341,2006,49.87870,37.93125,18.65987,-3.63581,-27.75665,-18.52988,7.76108,3.56109,-2.50351,...,32.38589,-32.75535,-61.05473,56.65182,15.29965,95.88193,-10.63242,12.96552,92.11633,10.88815
515342,2006,45.12852,12.65758,-38.72018,8.80882,-29.29985,-2.28706,-18.40424,-22.28726,-4.52429,...,-18.73598,-71.15954,-123.98443,121.26989,10.89629,34.62409,-248.61020,-6.07171,53.96319,-8.09364
515343,2006,44.16614,32.38368,-3.34971,-2.49165,-19.59278,-18.67098,8.78428,4.02039,-12.01230,...,67.16763,282.77624,-4.63677,144.00125,21.62652,-29.72432,71.47198,20.32240,14.83107,39.74909


Мы вывели кусок данных, чтобы понять, насколько они пригодны для работы без изменений. Здесь ясно, что сомнительно дальше с такими данными работать, потому что как минимум есть отрицательные значения, которые не отмасштабированы, кроме того еще сразу бросается в глаза совсем разная размерность, где-то видим реально большие числа, а где-то 0.075. Ясно, что будем скейлить.

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

Для выполнения данного задания (и всех последующих) предобработайте данные.

1. Зафиксируйте random_seed везде где только возможно. Вам предоставлена функция для этого, однако вы можете дополнить ее своими дополнениями
2. Обучите StandertScaler и предобработайте ваши данные. В следующих заданиях можете использовать другой scaler или вообще отказаться от него


In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    numpy.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [ ]:
from sklearn.metrics import mean_squared_error

def RMSE(y_true: numpy.ndarray, y_pred: numpy.ndarray) -> float:
    assert y_true.shape == y_pred.shape
    return mean_squared_error(y_true, y_pred, squared = False)

In [ ]:
import abc
import typing
from sklearn.preprocessing import StandardScaler

class BaseModel(abc.ABC):
    def preprocess(self, X: numpy.ndarray, y: typing.Optional[numpy.ndarray] = None):
        if (y is not None):
            # It is the train dataset
            y = y.reshape(-1, 1)
            self.x_scaler = StandardScaler().fit(X)
            self.y_scaler = StandardScaler().fit(y)
            X = self.x_scaler.transform(X)
            y = self.y_scaler.transform(y).flatten()
            return X, y
        else:
            # It is the test dataset
            return self.x_scaler.transform(X)

    def postprocess(self, y_pred: numpy.ndarray) -> numpy.ndarray:
        y_pred = y_pred.reshape(-1, 1)
        return self.y_scaler.inverse_transform(y_pred).flatten()

    @abc.abstractmethod
    def fit(self, X_train: numpy.ndarray, y_train: numpy.ndarray):
        raise NotImplementedError

    @abc.abstractmethod
    def predict(self, X_test: numpy.ndarray) -> numpy.ndarray:
        raise NotImplementedError

In [ ]:
from sklearn.linear_model import Ridge

class RidgeRegresseion(BaseModel):
    def __init__(self, alpha = 1.0):
        self.alpha = alpha

    def fit(self, X_train: numpy.ndarray, y_train: numpy.ndarray):
        X, y = self.preprocess(X_train, y_train)
        self.model = Ridge(self.alpha).fit(X, y)
        return self

    def predict(self, X_test: numpy.ndarray) -> numpy.ndarray:
      X = self.preprocess(X_test)
      y = self.model.predict(X)
      return self.postprocess(y)

model = RidgeRegresseion().fit(X_train, y_train)
display(RMSE(y_test, model.predict(X_test)))

9.510160820470436

Лучшая константа для RMSE это среднее, посчитаем значение метрики при нем

In [ ]:
from sklearn.linear_model import Ridge

class ConstantRegresseion(BaseModel):
    def fit(self, X_train: numpy.ndarray, y_train: numpy.ndarray):
        self.prediction = numpy.mean(y_train)
        return self

    def predict(self, X_test: numpy.ndarray) -> numpy.ndarray:
      return numpy.full(X_test.shape[0], self.prediction)

model = ConstantRegresseion().fit(X_train, y_train)
y_pred = model.predict(X_test)
assert numpy.all(y_pred == y_pred[0])
display(RMSE(y_test, y_pred))

10.85246390513634

## Задание 1. (максимум 10 баллов)

Закрепите свои знания о том, как pytorch работает с обратным распространением ошибки, проделав следующие шаги:

1. Создайте модель линейной регрессии, которая будет состоять только из одного Linear слоя.
2. Напишите цикл обучения вашей линейной регрессии. В нем реализуйте подсчет функции потерь, сделайте шаг градиентного спуска. Запрещено использовать готовые оптимизаторы и loss-функции из библиотеки pytorch. Для подсчета градиента воспользуйтесь методом backward.
3. Запустите обучение на 10 эпохах, после каждой проверяйте значение целевой метрики на тестовой выборке.
4. Выведите на экран графики метрики и значения функции потерь на тестовой и обучающей выборке.

В данном задании нет цели побить какой-то порог по метрике. Ваша задача - убедиться в том, что ваш рукописный цикл обучения работает. Для ускорения вычислений и обучения модели можете брать только срез данных, а не весь датасет.

In [ ]:
class TorchModel(BaseModel):
    def preprocess(self, X: numpy.ndarray, y: typing.Optional[numpy.ndarray] = None):
        if y is None:
            X = super().preprocess(X, y)
            X = torch.tensor(X, dtype = torch.float32)
            return X
        else:
            X, y = super().preprocess(X, y)
            X = torch.tensor(X, dtype = torch.float32)
            y = torch.tensor(y, dtype = torch.float32)
            return X, y

    def postprocess(self, y_pred: torch.Tensor) -> numpy.ndarray:
        return super().postprocess(y_pred.numpy())

In [ ]:
class MyTorchRegresseion(TorchModel):
    def __init__(self, epochs = 10, learning_rate = 1.0):
        self.epochs = epochs
        self.learning_rate = learning_rate

    def loss(self, y_true: torch.Tensor, y_pred: torch.Tensor) -> float:
        assert y_true.shape == y_pred.shape
        se = torch.sum(torch.square(y_pred - y_true))
        mse = torch.div(se, y_true.shape[0])
        return torch.sqrt(mse)

    def fit(self, X_train: numpy.ndarray, y_train: numpy.ndarray):
        self.model = torch.nn.Linear(X_train.shape[1], 1, bias = False)
        X, y = self.preprocess(X_train, y_train)
        for epoch in range(self.epochs):
            y_pred = self.model(X).squeeze(1)
            loss = self.loss(y, y_pred)
            loss.backward()
            with torch.no_grad():
                self.model.weight -= self.learning_rate * self.model.weight.grad
            self.model.weight.grad.zero_()
        return self

    def predict(self, X_test: numpy.ndarray) -> numpy.ndarray:
        with torch.no_grad():
            X = self.preprocess(X_test)
            y = self.model(X)
            return self.postprocess(y)

set_random_seed(42)
model = MyTorchRegresseion(10, 0.2).fit(X_train, y_train)
y_pred = model.predict(X_test)
display(RMSE(y_test, y_pred))

9.979489325704439

## Задание 2. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из предыдущего задания.



Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться сигнатурами функций, которые приведены ниже. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X.clone().detach()
        self.y = y.clone().detach()

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]

In [ ]:
class TorchRegresseion(TorchModel):
    def preprocess(self, X: numpy.ndarray, y: typing.Optional[numpy.ndarray] = None):
        if y is None:
            X = super().preprocess(X, y)
            return X.to(device)
        else:
            X, y = super().preprocess(X, y)
            return X.to(device), y.to(device)

    def postprocess(self, y_pred: torch.Tensor) -> numpy.ndarray:
        return super().postprocess(y_pred.cpu())

    def __init__(self):
        self.n_epochs = 21
        self.batch_size = 256

    def fit(self, X_train: numpy.ndarray, y_train: numpy.ndarray):
        X, y = self.preprocess(X_train, y_train)
        dataset = MyDataset(X, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size = self.batch_size, shuffle = True)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(X_train.shape[1], 60),
            torch.nn.BatchNorm1d(60),
            torch.nn.RReLU(),
            torch.nn.Linear(60, 35),
            torch.nn.BatchNorm1d(35),
            torch.nn.RReLU(),
            torch.nn.Linear(35, 15),
            torch.nn.BatchNorm1d(15),
            torch.nn.RReLU(),
            torch.nn.Linear(15, 1)
        ).to(device)
        # optimizer = torch.optim.SGD(self.model.parameters(), lr = 0.001, momentum = 0.5, weight_decay = 0.01)
        # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.975)
        optimizer = torch.optim.AdamW(self.model.parameters()) # 8.72 on GPU (21 epochs) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        for epoch in tqdm(range(self.n_epochs)):
            self.model.train()
            avg_test_loss = 0
            avg_test_loss_amount = 0
            for X, y in loader:
                optimizer.zero_grad()
                y_pred = self.model(X).squeeze(1)
                loss = torch.sqrt(torch.nn.functional.mse_loss(y_pred, y))
                loss.backward()
                optimizer.step()
                avg_test_loss += loss.item()
                avg_test_loss_amount += 1
            avg_test_loss /= avg_test_loss_amount
            # scheduler.step()

            y_pred = self.predict(X_test)
            print("Train: {}, Test: {}".format(avg_test_loss, RMSE(y_test, y_pred)))

        return self

    def predict(self, X_test: numpy.ndarray) -> numpy.ndarray:
        self.model.eval()
        with torch.no_grad():
            X = self.preprocess(X_test)
            y = self.model(X)
            return self.postprocess(y)

set_random_seed(42)
model = TorchRegresseion().fit(X_train, y_train)
y_pred = model.predict(X_test)
display(RMSE(y_test, y_pred))

  0%|          | 0/21 [00:00<?, ?it/s]

Train: 0.8195292128961345, Test: 8.909284107427402
Train: 0.7995333817949885, Test: 8.859873910564028
Train: 0.794809467745143, Test: 8.821203822659456
Train: 0.7905922080112608, Test: 8.799606738851454
Train: 0.7883492200974597, Test: 8.822990286021454
Train: 0.7863887053386002, Test: 8.818635927198532
Train: 0.7850019424232689, Test: 8.776967142088925
Train: 0.7827525994438209, Test: 8.761481817924935
Train: 0.7817835217674837, Test: 8.778497031599452
Train: 0.7806031747369577, Test: 8.760880645777036
Train: 0.7798439871528028, Test: 8.770369416309478
Train: 0.7787884514918654, Test: 8.770991583792803
Train: 0.778118775446157, Test: 8.78368433790044
Train: 0.7773495406307132, Test: 8.776910915421414
Train: 0.776803188831864, Test: 8.771984924791159
Train: 0.7767792578959307, Test: 8.774163005760402
Train: 0.7757119239692394, Test: 8.783833150093805
Train: 0.7749220118025281, Test: 8.753090440297536
Train: 0.7753251611396966, Test: 8.776671974099443
Train: 0.7742381892259548, Test: 8.

8.74518872961866

In [ ]:
# 8.970243432735815 - self.hidden_layer = 60
# 8.99191552125282 - self.hidden_layer = 45
# 8.98188805964392 - self.hidden_layer = 90
# 8.904352499264132 - optimizer = torch.optim.SGD(self.model.parameters(), lr = 0.15, momentum = 0.5, weight_decay = 0.001)

# 8.85
#    optimizer = torch.optim.SGD(self.model.parameters(), lr = 0.01, momentum = 0.5, weight_decay = 0.01)
#    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.975)

# 8.822353630832335 - optimizer = torch.optim.AdamW(self.model.parameters(), lr = 0.0001, weight_decay = 0.1)
# 8.811005475169399 - optimizer = torch.optim.AdamW(self.model.parameters(), lr = 0.0001, weight_decay = 0)

# 8.750570487322715 - 43 epochs
        self.model = torch.nn.Sequential(
            torch.nn.Linear(X_train.shape[1], 60),
            torch.nn.BatchNorm1d(60),
            torch.nn.RReLU(),
            torch.nn.Linear(60, 30),
            torch.nn.BatchNorm1d(30),
            torch.nn.RReLU(),
            torch.nn.Linear(30, 1)
        ).to(device)

        optimizer = torch.optim.AdamW(self.model.parameters(), lr = 0.001, weight_decay = 0)

## Задание 3. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.